## Imports

In [1]:
import datetime
import gc

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

In [3]:
cuda_use_gpus(0)

In [4]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [5]:
feature_list_id = 'oofp_lystdo_bi_lstm_magic'

In [6]:
RANDOM_SEED = 42

In [7]:
np.random.seed(RANDOM_SEED)

## Read Data

In [8]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [9]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [10]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [11]:
magic_feature_lists = [
    'magic_jturkewitz',
    'magic_stas_svd_150',
    'magic_stas_avito',
]

In [12]:
X_train_magic, X_test_magic, _ = load_feature_lists(magic_feature_lists)

In [13]:
X_train_magic = X_train_magic.values
X_test_magic = X_test_magic.values

In [14]:
scaler = StandardScaler()
scaler.fit(np.vstack([X_train_magic, X_test_magic]))
X_train_magic = scaler.transform(X_train_magic)
X_test_magic = scaler.transform(X_test_magic)

In [15]:
y_train = load(features_data_folder + 'y_train.pickle')

In [16]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [17]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101442 30


## Train Models & Compute Out-of-Fold Predictions

In [18]:
def create_model_question_branch():
    input_q = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    
    embedding_q = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )(input_q)

    timedist_q = TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu',
    ))(embedding_q)

    lambda_q = Lambda(
        lambda x: K.max(x, axis=1),
        output_shape=(EMBEDDING_DIM, )
    )(timedist_q)
    
    output_q = lambda_q
    return input_q, output_q

In [19]:
def create_model(params):    
    embedding_layer = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )
    lstm_layer = Bidirectional(LSTM(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
    ))

    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = lstm_layer(embedded_sequences_1)

    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = lstm_layer(embedded_sequences_2)

    magic_input = Input(shape=(X_train_magic.shape[-1], ))
    
    merged = concatenate([x1, y1, magic_input])
    merged = Dropout(params['dense_dropout_rate'])(merged)
    merged = BatchNormalization()(merged)

    merged = Dense(params['num_dense'], activation='relu')(merged)
    merged = Dropout(params['dense_dropout_rate'])(merged)
    merged = BatchNormalization()(merged)

    output = Dense(1, activation='sigmoid')(merged)

    model = Model(
        inputs=[sequence_1_input, sequence_2_input, magic_input],
        outputs=output
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
    )

    return model

In [20]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [21]:
NUM_FOLDS = 5
NUM_EPOCHS = 200

In [22]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [23]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [24]:
y_test_oofp = np.zeros((len(X_test_q1), NUM_FOLDS))

In [25]:
model_params = {
    'dense_dropout_rate': 0.07512852175097123,
    'lstm_dropout_rate': 0.3320541002991107,
    'num_dense': 130,
    'num_lstm': 333
}

In [26]:
def predict(model, X_q1, X_q2, X_magic):
    y1 = model.predict(
        [X_q1, X_q2, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    y2 = model.predict(
        [X_q2, X_q1, X_magic],
        batch_size=1024,
        verbose=1
    ).reshape(-1)
    
    return (y1 + y2) / 2

In [27]:
%%time

for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
    X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])
    X_fold_train_magic = np.vstack([X_train_magic[ix_train], X_train_magic[ix_train]])

    X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
    X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])
    X_fold_val_magic = np.vstack([X_train_magic[ix_val], X_train_magic[ix_val]])

    y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
    y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        [X_fold_train_q1, X_fold_train_q2, X_fold_train_magic], y_fold_train,
        validation_data=([X_fold_val_q1, X_fold_val_q2, X_fold_val_magic], y_fold_val),

        batch_size=1024,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val] = predict(model, X_train_q1[ix_val], X_train_q2[ix_val], X_train_magic[ix_val])
    y_test_oofp[:, fold_num] = predict(model, X_test_q1, X_test_q2, X_test_magic)
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_train_magic
    del X_fold_val_q1
    del X_fold_val_q2
    del X_fold_val_magic
    del model
    gc.collect()


Fitting fold 1 of 5

Train on 646862 samples, validate on 161718 samples
Epoch 1/200
646862/646862 [==============================] - 203s - loss: 0.3850 - acc: 0.8256 - val_loss: 0.3318 - val_acc: 0.8563
Epoch 2/200
646862/646862 [==============================] - 205s - loss: 0.3210 - acc: 0.8554 - val_loss: 0.3022 - val_acc: 0.8661
Epoch 3/200
646862/646862 [==============================] - 204s - loss: 0.2952 - acc: 0.8671 - val_loss: 0.2943 - val_acc: 0.8671
Epoch 4/200
646862/646862 [==============================] - 203s - loss: 0.2779 - acc: 0.8752 - val_loss: 0.2763 - val_acc: 0.8766
Epoch 5/200
646862/646862 [==============================] - 203s - loss: 0.2636 - acc: 0.8816 - val_loss: 0.2799 - val_acc: 0.8732
Epoch 6/200
646862/646862 [==============================] - 204s - loss: 0.2510 - acc: 0.8875 - val_loss: 0.2710 - val_acc: 0.8808
Epoch 7/200
646862/646862 [==============================] - 204s - loss: 0.2401 - acc: 0.8931 - val_loss: 0.2905 - val_acc: 0.8691
Ep

646864/646864 [==============================] - 200s - loss: 0.2760 - acc: 0.8761 - val_loss: 0.2844 - val_acc: 0.8704
Epoch 5/200
646864/646864 [==============================] - 201s - loss: 0.2613 - acc: 0.8833 - val_loss: 0.2819 - val_acc: 0.8719
Epoch 6/200
646864/646864 [==============================] - 206s - loss: 0.2486 - acc: 0.8888 - val_loss: 0.2769 - val_acc: 0.8746
Epoch 7/200
646864/646864 [==============================] - 207s - loss: 0.2383 - acc: 0.8942 - val_loss: 0.2749 - val_acc: 0.8774
Epoch 8/200
646864/646864 [==============================] - 207s - loss: 0.2277 - acc: 0.8988 - val_loss: 0.2936 - val_acc: 0.8684
Epoch 9/200
646864/646864 [==============================] - 207s - loss: 0.2195 - acc: 0.9029 - val_loss: 0.2847 - val_acc: 0.8716
Epoch 10/200
646864/646864 [==============================] - 205s - loss: 0.2115 - acc: 0.9070 - val_loss: 0.2830 - val_acc: 0.8743
Epoch 11/200
646864/646864 [==============================] - 209s - loss: 0.2047 - acc

646866/646866 [==============================] - 212s - loss: 0.3206 - acc: 0.8550 - val_loss: 0.3088 - val_acc: 0.8606
Epoch 3/200
646866/646866 [==============================] - 212s - loss: 0.2955 - acc: 0.8673 - val_loss: 0.2974 - val_acc: 0.8660
Epoch 4/200
646866/646866 [==============================] - 212s - loss: 0.2782 - acc: 0.8751 - val_loss: 0.2789 - val_acc: 0.8756
Epoch 5/200
646866/646866 [==============================] - 212s - loss: 0.2637 - acc: 0.8818 - val_loss: 0.2763 - val_acc: 0.8768
Epoch 6/200
646866/646866 [==============================] - 212s - loss: 0.2504 - acc: 0.8882 - val_loss: 0.2768 - val_acc: 0.8761
Epoch 7/200
646866/646866 [==============================] - 212s - loss: 0.2399 - acc: 0.8932 - val_loss: 0.2715 - val_acc: 0.8804
Epoch 8/200
646866/646866 [==============================] - 212s - loss: 0.2294 - acc: 0.8982 - val_loss: 0.2965 - val_acc: 0.8687
Epoch 9/200
646866/646866 [==============================] - 212s - loss: 0.2212 - acc: 

In [28]:
cv_score = log_loss(y_train, y_train_oofp)
print('CV score:', cv_score)

CV score: 0.271096058182


## Save feature names

In [29]:
feature_names = [
    'oofp_lystdo_bi_lstm_magic',
]

In [30]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save Train features

In [31]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [32]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [33]:
y_test_oofp_mean = np.mean(y_test_oofp, axis=1).reshape((-1, 1))

In [34]:
save(y_test_oofp_mean, features_data_folder + f'X_test_{feature_list_id}.pickle')